# Initial setup

In [1]:
import asyncio
import os
import PyPDF2
import sys

from aiofile import AIOFile
from aiohttp import ClientSession
from bs4 import Comment
from src.data_obtaining.webpage_interaction import get_webpage_as_bs, download_document_and_save

# Constantes

In [2]:
URLS = ['http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18']

In [3]:
LEVELS = {
    'Educación Inicial': '1',
    'Educación Primaria': '2',
    'Educación Secundaria': '3'
}

In [4]:
ACCEPTED_LANGUAGE = 'ES (Español)'

In [5]:
ACCEPTED_AREAS = ['Formación Ciudadana y Cívica', 
                  'Arte', 
                  'Comunicación', 
                  'CTA', 
                  'Educación Física', 
                  'Educación para el Trabajo',
                  'Historia, Geografía y Economía']
                  # 'Matemática']

In [6]:
PROJECT_PATH = '/home/hans/Documentos/Tesis_Chatbot'

# Helper functions

In [7]:
async def obtain_and_save_document(session: ClientSession, url: str) -> None:
    """
    This function looks for the link to download a document in "perueduca.com"

    Parameters:
    session (ClientSession): The async session used to obtain the webpage
    url (str): The url of the webpage to convert

    Returns:
    None
    """
    try:
        soup = await get_webpage_as_bs(session, url)
        document_information_table = soup.find('div', id='ficha-catalogo').find_all('tr')
        document_language = document_information_table[7].find('td').text.strip()
        document_category = document_information_table[3].find('td').text.strip()
        if (document_language == ACCEPTED_LANGUAGE and document_category in ACCEPTED_AREAS):
            document_level = LEVELS[document_information_table[1].find('td').text.strip()]
            download_link = soup.find('a', id='hf_iframe')['href']
            document_name = download_link.split('/')[-1]
            storage_directory = PROJECT_PATH + f'/data/raw/pdf/{document_level}/{document_name}'
            await download_document_and_save(session, download_link, storage_directory)
            print(f'Document {document_name} downloaded successfully')
    except Exception as e:
        raise e

In [8]:
async def obtain_texts_from_perueduca_webpage(session: ClientSession, url: str) -> None:
    """
    This function traverses through an entire web page of peru educa, searching for documents to download.
    It's a recursive function.
    
    This function begins with obtaining the webpage as a beautiful soup object, then, it proceeds to find
    all the available document sections in this page so it can download them. After that, it goes to the
    next page and does the same process again.

    Parameters:
    session (ClientSession): The async session used to obtain the webpage
    url (str): The url of the webpage to convert

    Returns:
    None
    """
    try:
        soup = await get_webpage_as_bs(session, url)
        texts_information = soup.find_all('div', class_='box-result')
        if (len(texts_information) == 0): # No more texts in this page. Extraction process finished
            return
        else: # More texts still available
            for text in texts_information:
                text_link = text.find('a')['href']
                await obtain_and_save_document(session, text_link)
            # There are still more pages to look documents into
            next_page_button = soup.find('li', class_='next')
            next_page_url = next_page_button.find('a')['href']
            await obtain_texts_from_perueduca_webpage(session, next_page_url)
    except Exception as e:
        raise e


In [12]:
async def convert_pdfs_to_txt_files() -> None:
    """
    This function scans all the files we just downloaded and converts them to a .txt format for easier processing
    """
    # levels = ['1', '2', '3']
    levels = [3]
    for level in levels:
        documents = [file_name for file_name in os.listdir(PROJECT_PATH + f'/data/raw/pdf/{level}') if '.pdf' in file_name]
        documents.sort()
        for document in documents:
            try:
                print(f'Converting {document} from pdf to txt')
                with open(PROJECT_PATH + f'/data/raw/pdf/{level}/{document}', 'rb') as pdf_file:
                    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                    text = ''.join([pdf_reader.getPage(page).extractText() for page in range(pdf_reader.numPages)]) 
                    with open(PROJECT_PATH + f'/data/raw/txt/{level}/{document}' + document.replace('.pdf', '.txt'), 'w') as text_file:
                        text_file.write(text)
                print(f'Text {document} converted from pdf to txt')
            except Exception as e:
                raise e
                continue

In [13]:
async def main() -> None:
    """
    The main function obtains the documents as pdfs and then converts them to text file format
    """
    """try:
        session = ClientSession()
        for url in URLS:
            await obtain_texts_from_perueduca_webpage(session, url)
        await session.close()
    except Exception as e:
        await session.close()"""
    await convert_pdfs_to_txt_files()

In [14]:
await main()

Converting 20_leguas_viaje_submarino.pdf from pdf to txt
Text 20_leguas_viaje_submarino.pdf converted from pdf to txt
Converting abel_sanchez.pdf from pdf to txt
Text abel_sanchez.pdf converted from pdf to txt
Converting agua_suelo.pdf from pdf to txt
Text agua_suelo.pdf converted from pdf to txt
Converting amistad_funesta.pdf from pdf to txt
Text amistad_funesta.pdf converted from pdf to txt
Converting antologia_leer_escribir.pdf from pdf to txt
Text antologia_leer_escribir.pdf converted from pdf to txt
Converting arguedas.pdf from pdf to txt
Text arguedas.pdf converted from pdf to txt
Converting aritmetica.pdf from pdf to txt
Text aritmetica.pdf converted from pdf to txt
Converting arte.pdf from pdf to txt
Text arte.pdf converted from pdf to txt
Converting arte_digital_interactividad.pdf from pdf to txt
Text arte_digital_interactividad.pdf converted from pdf to txt
Converting augusto_leguia.pdf from pdf to txt
Text augusto_leguia.pdf converted from pdf to txt
Converting aventuras_rob

Text tres_mosqueteros.pdf converted from pdf to txt
Converting usos_internet.pdf from pdf to txt
Text usos_internet.pdf converted from pdf to txt
Converting viaje_centro_tierra.pdf from pdf to txt
Text viaje_centro_tierra.pdf converted from pdf to txt
Converting viajes_gulliver.pdf from pdf to txt
Text viajes_gulliver.pdf converted from pdf to txt
Converting vida_lazarillo_tormes.pdf from pdf to txt
Text vida_lazarillo_tormes.pdf converted from pdf to txt
Converting vuelta_mundo_80.pdf from pdf to txt
Text vuelta_mundo_80.pdf converted from pdf to txt
Converting web_2.pdf from pdf to txt
Text web_2.pdf converted from pdf to txt
